In [1]:
%matplotlib inline
# Load all necessary packages
import sys
import pandas as pd
# Import IBM's AI Fairness tooolbox
from aif360.datasets import BinaryLabelDataset
from aif360.metrics  import BinaryLabelDatasetMetric
from aif360.metrics  import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.lfr import LFR
# Import scikit-learn core slibraries
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned German-Cleaned dataset
German_df = pd.read_csv('./input/German-Cleaned.csv')

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'Gender'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'CreditStatus'
fav_label           = 1 # Good Credit status
unfav_label         = 0 # Bad Credit Status
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females

In [4]:
# Create a Binary Label Dataset to use with AIF360 APIs
X = German_df.drop(lab,axis=1)
y = German_df[lab]
German_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [5]:
# Create train and test datasets
German_train_df, German_test_df = German_bld.split([0.8], shuffle=True, seed=101)

In [6]:
scaler = MinMaxScaler(copy=False)
German_train_df.features = scaler.fit_transform(German_train_df.features)
German_test_df.features  = scaler.fit_transform(German_test_df.features)

In [7]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
German_train, d = German_train_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
German_test, d = German_test_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_train = German_train.drop(lab,axis=1)
y_train = German_train[lab]
X_test = German_test.drop(lab,axis=1)
y_test = German_test[lab]
biasedLogModel = LogisticRegression(random_state=101)
biasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
biasedLogModel.fit(X_train, y_train) 
biasedRfcModel.fit(X_train, y_train) 
print(f"Logistic regression validation accuracy: {biasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {biasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.72
Random Forest       validation accuracy: 0.765


In [8]:
# Create the binary label dataset metric class for the train dataset
metric_train_bld = BinaryLabelDatasetMetric(German_train_df, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())
print('Number of instances           :', metric_train_bld.num_instances())
print("Base Rate                     :%f" % metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
print('# of positives(privileged)    :', metric_train_bld.num_positives(privileged=True))
print('# of positives(non-privileged):', metric_train_bld.num_positives(privileged=False))
print('Total positive instances"     :', metric_train_bld.num_positives(privileged=True)+metric_train_bld.num_positives(privileged=False))
print('# of negatives(privileged)    :', metric_train_bld.num_negatives(privileged=True))
print('# of negatives(non-privileged):', metric_train_bld.num_negatives(privileged=False))
print('Total negative instances"     :', metric_train_bld.num_negatives(privileged=True)+metric_train_bld.num_negatives(privileged=False))
display(Markdown("#### Biased training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())

#### Biased training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.092000
Number of instances           : 800.0
Base Rate                     :0.691250
Consistency                   : [0.7055]
Disparate Impact              : 0.8722222222222222
Mean Difference               : -0.09199999999999997
Statistical Parity Difference : -0.09199999999999997
# of positives(privileged)    : 396.0
# of positives(non-privileged): 157.0
Total positive instances"     : 553.0
# of negatives(privileged)    : 154.0
# of negatives(non-privileged): 93.0
Total negative instances"     : 247.0


#### Biased training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.092000


In [9]:
# Fit the Learning Fair Representations on the biased training data
print('Interval : Optimization objective value for the interval')
TR = LFR(unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups, verbose=1, seed=101)
TR = TR.fit(German_train_df)

Interval : Optimization objective value for the interval
250 520.5934717667799
500 507.94546596469763
750 500.26258600718523
1000 487.8615570680949
1250 479.6683181588537
1500 471.0245081739862
1750 467.2477652935682
2000 461.80876024799176
2250 456.09039120052665
2500 455.3834676146122
2750 453.89696210469333
3000 452.84732425421873
3250 451.9724476161422
3500 451.1493924850815
3750 450.3044399365166
4000 449.050321176735
4250 446.7635202613466
4500 446.6616512259222
4750 446.16890815018337
5000 445.987740406953


In [10]:
# Transform training data and align features
German_train_lfr = TR.transform(German_train_df)

In [11]:
# Determine the transformed model accuracy for Logistic Regression and Random Forest Classifiers
# Convert the scaled Binary Labelled Dataset to a pandas dataframe for consistency 
German_train, d = German_train_lfr.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_train = German_train.drop(lab,axis=1)
y_train = German_train[lab]
debiasedLogModel = LogisticRegression(random_state=101)
debiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
debiasedLogModel.fit(X_train, y_train) 
debiasedRfcModel.fit(X_train, y_train) 
print(f"Logistic regression validation accuracy: {debiasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {debiasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.725
Random Forest       validation accuracy: 0.715


In [12]:
display(Markdown("#### Transformed training dataset"))
from sklearn.metrics import classification_report
thresholds = [0.1, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7]
for threshold in thresholds:
    
    # Transform training data and align features
    German_train_lfr = TR.transform(German_train_df,threshold=threshold)
    metric_train_lfr = BinaryLabelDatasetMetric(German_train_lfr, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    print("Classification threshold = %f" % threshold)
    #print(classification_report(dataset_orig_train.labels, dataset_transf_train.labels))
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_lfr.mean_difference())

#### Transformed training dataset

Classification threshold = 0.100000
Difference in mean outcomes between unprivileged and privileged groups = 0.000000
Classification threshold = 0.300000
Difference in mean outcomes between unprivileged and privileged groups = -0.002545
Classification threshold = 0.350000
Difference in mean outcomes between unprivileged and privileged groups = -0.000364
Classification threshold = 0.400000
Difference in mean outcomes between unprivileged and privileged groups = 0.012364
Classification threshold = 0.500000
Difference in mean outcomes between unprivileged and privileged groups = 0.022909
Classification threshold = 0.600000
Difference in mean outcomes between unprivileged and privileged groups = 0.010545
Classification threshold = 0.700000
Difference in mean outcomes between unprivileged and privileged groups = -0.038909


In [13]:
print('THIS IS WHAT THE AIF CODE PRODUCES FROM THE LAST CLASSIFICATION THRESHOLD RUN ABOVE')
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_lfr.mean_difference())
print('Number of instances           :', metric_train_lfr.num_instances())
print("Base Rate                     :%f" % metric_train_lfr.base_rate())
print('Consistency                   :', metric_train_lfr.consistency())
print('Disparate Impact              :', metric_train_lfr.disparate_impact())
print('Mean Difference               :', metric_train_lfr.mean_difference())
print('Statistical Parity Difference :', metric_train_lfr.statistical_parity_difference()) 
print('# of positives(privileged)    :', metric_train_lfr.num_positives(privileged=True))
print('# of positives(non-privileged):', metric_train_lfr.num_positives(privileged=False))
print('Total positive instances"     :', metric_train_lfr.num_positives(privileged=True)+metric_train_lfr.num_positives(privileged=False))
print('# of negatives(privileged)    :', metric_train_lfr.num_negatives(privileged=True))
print('# of negatives(non-privileged):', metric_train_lfr.num_negatives(privileged=False))
print('Total negative instances"     :', metric_train_lfr.num_negatives(privileged=True)+metric_train_lfr.num_negatives(privileged=False))
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())

THIS IS WHAT THE AIF CODE PRODUCES FROM THE LAST CLASSIFICATION THRESHOLD RUN ABOVE
Difference in mean outcomes between unprivileged and privileged groups = -0.038909
Number of instances           : 800.0
Base Rate                     :0.518750
Consistency                   : [0.97725]
Disparate Impact              : 0.9267123287671233
Mean Difference               : -0.03890909090909089
Statistical Parity Difference : -0.03890909090909089
# of positives(privileged)    : 292.0
# of positives(non-privileged): 123.0
Total positive instances"     : 415.0
# of negatives(privileged)    : 258.0
# of negatives(non-privileged): 127.0
Total negative instances"     : 385.0


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.092000


In [14]:
display(Markdown("#### Individual fairness metrics"))
print("Consistency of labels in transformed training dataset= %f" %metric_train_lfr.consistency())
print("Consistency of labels in original training dataset= %f" %metric_train_bld.consistency())

#### Individual fairness metrics

Consistency of labels in transformed training dataset= 0.977250
Consistency of labels in original training dataset= 0.705500


In [15]:
# Compare the original and transformed datasets
#Convert the returned Binary Labelled Dataset to a pandas dataframe 
German_orig_df, d = German_train_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
German_lfr_df, d = German_train_lfr.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
# Check whether the transform on the original dataset has worked. 
# A false means that the dataset is transformed.
German_lfr_df.equals(German_df)

False

In [16]:
German_lfr_df.head(3)

,NumMonths,CreditHistory,Purpose,CreditAmount,Savings,EmployDuration,PayBackPercent,Gender,Debtors,ResidenceDuration,Collateral,OtherPayBackPlan,Property,ExistingCredit,Job,Dependents,Telephone,Foreignworker,CreditStatus
545,0.853476,0.742106,0.326361,0.397229,0.361051,0.714082,1.025977,1.0,0.010094,0.659228,0.554914,0.256358,0.444575,0.015676,0.188846,0.329984,0.839221,-0.289778,0.0
298,-0.153830,-0.133537,0.076879,-0.275152,0.272333,0.272349,0.574971,1.0,-0.045511,0.728774,0.557201,-0.261114,0.209190,0.064675,-0.012957,-0.058200,0.424140,0.005392,1.0
109,-0.258166,-0.212034,0.083563,-0.281475,0.272554,0.216912,0.514884,1.0,-0.042815,0.693015,0.562503,-0.246121,0.191181,0.106810,0.000222,-0.073863,0.358425,0.082273,1.0


In [17]:
German_orig_df.head(3)

,NumMonths,CreditHistory,Purpose,CreditAmount,Savings,EmployDuration,PayBackPercent,Gender,Debtors,ResidenceDuration,Collateral,OtherPayBackPlan,Property,ExistingCredit,Job,Dependents,Telephone,Foreignworker,CreditStatus
545,0.294118,0.50,0.333333,0.059591,0.25,0.75,1.000000,1.0,0.0,0.333333,0.0,0.0,0.5,0.333333,0.0,1.0,1.0,0.0,0.0
298,0.205882,0.25,0.222222,0.124629,0.25,0.25,0.666667,1.0,0.0,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0
109,0.147059,0.25,0.555556,0.063827,0.50,0.00,0.000000,1.0,0.0,0.333333,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0


In [18]:
German_orig_df   = German_orig_df.reset_index(drop=True)
German_lfr_df = German_lfr_df.reset_index(drop=True)
GermanBool   = (German_orig_df != German_lfr_df).stack()  # Create Frame of comparison booleans
Germandiff   = pd.concat([German_orig_df.stack()[GermanBool], German_lfr_df.stack()[GermanBool]], axis=1)
Germandiff.columns=["German_df", "German_lfr"]
print(Germandiff)

                       German_df  German_lfr
0   NumMonths           0.294118    0.853476
    CreditHistory       0.500000    0.742106
    Purpose             0.333333    0.326361
    CreditAmount        0.059591    0.397229
    Savings             0.250000    0.361051
    EmployDuration      0.750000    0.714082
    PayBackPercent      1.000000    1.025977
    Debtors             0.000000    0.010094
    ResidenceDuration   0.333333    0.659228
    Collateral          0.000000    0.554914
    OtherPayBackPlan    0.000000    0.256358
    Property            0.500000    0.444575
    ExistingCredit      0.333333    0.015676
    Job                 0.000000    0.188846
    Dependents          1.000000    0.329984
    Telephone           1.000000    0.839221
    Foreignworker       0.000000   -0.289778
1   NumMonths           0.205882   -0.153830
    CreditHistory       0.250000   -0.133537
    Purpose             0.222222    0.076879
    CreditAmount        0.124629   -0.275152
    Saving

In [19]:
## PCA Analysis of consitency

In [20]:
# At this stage the transformed dataframe will have the last threshold encountered!
print('this is the threshold being used :',threshold)
feat_cols = German_train_df.feature_names

orig_df = pd.DataFrame(German_train_df.features,columns=feat_cols)
orig_df['label'] = German_train_df.labels
orig_df['label'] = orig_df['label'].apply(lambda i: str(i))

transf_df = pd.DataFrame(German_train_lfr.features,columns=feat_cols)
transf_df['label'] = German_train_lfr.labels
transf_df['label'] = transf_df['label'].apply(lambda i: str(i))

this is the threshold being used : 0.7


In [21]:
from sklearn.decomposition import PCA

orig_pca = PCA(n_components=3)
orig_pca_result = orig_pca.fit_transform(orig_df[feat_cols].values)

orig_df['pca-one'] = orig_pca_result[:,0]
orig_df['pca-two'] = orig_pca_result[:,1] 
orig_df['pca-three'] = orig_pca_result[:,2]

display(Markdown("#### Original training dataset"))
print('Explained variation per principal component:')
print(orig_pca.explained_variance_ratio_)

#### Original training dataset

Explained variation per principal component:
[0.15576645 0.13448264 0.10111426]


In [22]:
transf_pca = PCA(n_components=3)
transf_pca_result = transf_pca.fit_transform(transf_df[feat_cols].values)

transf_df['pca-one'] = transf_pca_result[:,0]
transf_df['pca-two'] = transf_pca_result[:,1] 
transf_df['pca-three'] = transf_pca_result[:,2]

display(Markdown("#### Transformed training dataset"))
print('Explained variation per principal component:')
print(transf_pca.explained_variance_ratio_)

#### Transformed training dataset

Explained variation per principal component:
[0.71598891 0.13888389 0.13536777]


In [23]:
# Test whether we can predict the Sensitive variable from the transformed training dataset
X_train = German_lfr_df.drop(protected_attr,axis=1)
y_train = German_lfr_df[protected_attr]
debiasedLogModel = LogisticRegression(random_state=101)
debiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
debiasedLogModel.fit(X_train, y_train) 
debiasedRfcModel.fit(X_train, y_train) 
# Now test whether we can predict Gender from the test dataset
German_test, d = German_test_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_test = German_test.drop(protected_attr,axis=1)
y_test = German_test[protected_attr]
print(f"Logistic regression validation accuracy: {debiasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {debiasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.7
Random Forest       validation accuracy: 0.695


In [24]:
###Load, clean up original test data and compute metric

In [25]:
display(Markdown("#### Testing Dataset shape"))
print(German_test_df.features.shape)

metric_test_bld = BinaryLabelDatasetMetric(German_test_df, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test_bld.mean_difference())

#### Testing Dataset shape

(200, 18)


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.002381


In [26]:
###Transform test data and compute metric

In [27]:
German_test_lfr = TR.transform(German_test_df, threshold=threshold)
metric_test_lfr = BinaryLabelDatasetMetric(German_test_lfr, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

In [28]:
print("Consistency of labels in tranformed test dataset= %f" %metric_test_lfr.consistency())

Consistency of labels in tranformed test dataset= 0.972000


In [29]:
print("Consistency of labels in original test dataset= %f" %metric_test_bld.consistency())

Consistency of labels in original test dataset= 0.711000


In [30]:
def check_algorithm_success():
    """Transformed dataset consistency should be greater than original dataset."""
    assert metric_test_lfr.consistency() > metric_test_bld.consistency(), "Transformed dataset consistency should be greater than original dataset."

print(check_algorithm_success())

None
